### Предобработка данных

In [2]:
# read file '../data/ostap.txt (only odd lines) and save to a list

with open('../data/ostap.txt') as f:
    lines = f.readlines()

odd_lines = [line for i, line in enumerate(lines) if i % 2 == 0]

In [4]:
len(odd_lines)

113

In [3]:
odd_lines[0]

'25 монгольских тугриков\n'

#### Генерация вопросов

In [2]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

In [10]:
from dotenv import load_dotenv

load_dotenv()

True

#### Пример генерации

In [91]:
label_template = """Вы — AI, задача которого создать датасет для обучения retrieval чат-бота, отвечающего как конкретный персонаж (Остап Бендер из 12 стульев).

Входные данные:
- Цитата персонажа: "{quote}"

Инструкции:
1. Мне нужно сгенерировать обычный короткий вопрос или просто предложение пользователя, который могут задать чат-бота, Например "Как дела?" или "Что нового?", "Понятно". 
2. Сгенерировать 10 вариантов вопросов (или фраз), которые могут написать пользователи чат-бота. Не обязательно вопрос должен быть прямым, главное чтобы он был вопросительным. Ответ должен быть смешным ответом персонажа на вопрос или фразу.
3. Не используй слова из вопросов (Например, если цитата про коня, не надо всегда спрашивать про коня, можно спросить про спорт (например шахматы)). Просто нужные обычные вопросы на русском языке.
4. Вопросы или фразы должны быть разные, но по тематике цитаты.
5. На каждый вопрос, сгенерируй ответ похожий на представленную цитату персонажа (каждый раз разные ответ).
5. Вывести в формате JSON.

Пример входных данных:
{{
  "quote": "'А что, разве я похож на человека, у которого могут быть родственники?\n'"
}}
Пример структуры вывода:
```json
{{
  "user_1": "У тебя есть родственники?",
  "answer_1": "А что, разве я похож на человека, у которого могут быть родственники?",
  "user_2": "Как твои дела?",
  "answer_2": "А что, разве я похож на человека, у которого могут быть дела?",
  ...
  "user_10": "Хорошая сегодня погода!"
  "answer_10": "А что, разве я похож на человека, который интересуется погодой?"
}}
"""



label_prompt = ChatPromptTemplate.from_template(label_template)
llm = ChatOpenAI(temperature=0.1, model="gpt-4o-mini")

label_chain = label_prompt | llm | JsonOutputParser()

In [95]:
example = 8

output = label_chain.invoke({"quote": odd_lines[example]})

In [96]:
odd_lines[example]

'Ах, вы думали? Вы, значит, иногда думаете? Вы мыслитель. Как ваша фамилия, мыслитель? Спиноза? Жан-Жак Руссо? Марк Аврелий?\n'

In [97]:
output

{'user_1': 'Как ты себя чувствуешь?',
 'answer_1': 'Ах, вы думали, что я чувствую? Вы, значит, иногда чувствуете? Как ваша фамилия, чувствующий? Гегель? Ницше? Сократ?',
 'user_2': 'Что ты думаешь о жизни?',
 'answer_2': 'Ах, вы думали, что я думаю о жизни? Вы, значит, иногда думаете? Как ваша фамилия, мыслитель? Платон? Кант? Гераклит?',
 'user_3': 'Какой у тебя любимый фильм?',
 'answer_3': 'Ах, вы думали, что у меня есть любимый фильм? Вы, значит, иногда смотрите? Как ваша фамилия, зритель? Тарковский? Бунюэль? Феллини?',
 'user_4': 'Какой у тебя хобби?',
 'answer_4': 'Ах, вы думали, что у меня есть хобби? Вы, значит, иногда увлекаетесь? Как ваша фамилия, увлеченный? Бах? Моцарт? Шопен?',
 'user_5': 'Что ты думаешь о спорте?',
 'answer_5': 'Ах, вы думали, что я думаю о спорте? Вы, значит, иногда размышляете? Как ваша фамилия, спортсмен? Пеле? Месси? Кличко?',
 'user_6': 'Какой у тебя любимый цвет?',
 'answer_6': 'Ах, вы думали, что у меня есть любимый цвет? Вы, значит, иногда выбира

### Генерация примеров для всех цитат

In [99]:
from tqdm import tqdm

data = []

for i in tqdm(range(len(odd_lines))):
    output = label_chain.invoke({"quote": odd_lines[i]})
    data.append(output)

100%|██████████| 113/113 [13:43<00:00,  7.29s/it]


In [100]:
# write to json file
with open('../data/ostap.json', 'w') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [3]:
### check the file
with open('../data/ostap.json') as f:
    data = json.load(f)

#### Собираем датасет

In [5]:
# create dataset for training question-answering model, columns: question, answer
qa_data = []

for item in data:
    for i in range(1, 11):
        qa_data.append({'question': item[f'user_{i}'], 'answer': item[f'answer_{i}']})



In [6]:
len(qa_data)

1130

In [7]:
# divide to train and test datasets
import random

random.seed(42)

random.shuffle(qa_data)

train_size = int(0.8 * len(qa_data))

train_data = qa_data[:train_size]
test_data = qa_data[train_size:]

len(train_data), len(test_data)

(904, 226)

In [9]:
# save to csv files
import pandas as pd

train_df = pd.DataFrame(train_data)

train_df.to_csv('../data/ostap_train.csv', index=False)

test_df = pd.DataFrame(test_data)

test_df.to_csv('../data/ostap_test.csv', index=False)

In [10]:
from datasets import Dataset, DatasetDict


# Convert DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
test_dataset = Dataset.from_pandas(test_df, preserve_index=False)

# Combine into a DatasetDict
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset})

In [13]:
# Push to Hugging Face Hub
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv('../.env')

# Log in to Hugging Face account
login(token=os.getenv('HF_KEY'))  # Replace with your HF token or use an environment variable

# Specify repository name
repo_name = "fitlemon/ostap-phrases"  

# Push to the Hugging Face Hub
dataset_dict.push_to_hub(repo_name, token=os.getenv('HF_KEY'))

print(f"Dataset pushed to the Hugging Face Hub under {repo_name}")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset pushed to the Hugging Face Hub under fitlemon/ostap-phrases


In [14]:
from datasets import load_dataset


# Load dataset from Hugging Face Hub
dataset = load_dataset("fitlemon/ostap-phrases")

# Access train/test splits
train_data = dataset["train"]
test_data = dataset["test"]

README.md:   0%|          | 0.00/404 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/51.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/904 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/226 [00:00<?, ? examples/s]

In [17]:
# concat Datasets
from datasets import concatenate_datasets

dataset = concatenate_datasets([train_data, test_data])

dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 1130
})